<a href="https://colab.research.google.com/github/resppiano/GH_collab/blob/main/cancellationroster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime
from PIL import Image
import pytesseract
import cv2
import re  # Regular expressions for cleaning names

# Add the path to the Tesseract executable if it's not in your PATH
# pytesseract.pytesseract.tesseract_cmd = r'C:\path\to\tesseract.exe'

def get_current_date():
    """
    Get the current date in a readable format.
    Returns:
    str: The current date as a string.
    """
    return datetime.datetime.now().strftime("%Y-%m-%d")

def find_next_to_cancel(working_therapists, therapists):
    """
    Find the next therapist to cancel who is actually working.
    """
    for therapist in therapists:
        if therapist in working_therapists:
            return therapist
    return None

def adjust_for_sick_call(sick_therapist, therapists):
    """
    Adjust the cancellation list to account for a therapist calling out sick.
    """
    if sick_therapist in therapists:
        therapists.remove(sick_therapist)
        therapists.insert(1, sick_therapist)

def update_cancellation_list(working_therapists, therapists, sick_therapist=None, requested_cancellation=None):
    """
    Update the cancellation list based on who is working, handle sick calls and requested cancellations.
    """
    cancellation_reason = ""
    if sick_therapist and sick_therapist in therapists:
        adjust_for_sick_call(sick_therapist, therapists)
        cancellation_reason = "call out sick"
    elif requested_cancellation and requested_cancellation in therapists:
        adjust_for_sick_call(requested_cancellation, therapists)
        cancellation_reason = "requested cancellation"
    else:
        cancellation_reason = "it was the therapist's turn for cancellation"

    next_to_cancel = find_next_to_cancel(working_therapists, therapists)
    if next_to_cancel:
        therapists.append(therapists.pop(0))  # Rotate the list

    cancellation_date = get_current_date()
    cancellation_info = f"{next_to_cancel} (Cancelled on {cancellation_date}, Reason: {cancellation_reason})"
    return cancellation_info, therapists

def extract_text_from_image(image_path):
    """
    Extract text from an image using OCR (Tesseract).
    """
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray)
    return text

def parse_extracted_text(extracted_text):
    """
    Parse the extracted text to get a list of therapist names.
    """
    # Split the text by commas and strip whitespace
    names = [name.strip() for name in extracted_text.split(',')]
    return names

def clean_names(names, known_names=None):
    """
    Clean the list of names by removing non-alphabetic characters and correcting common OCR errors.
    Optionally validate against a list of known names.
    """
    cleaned_names = []
    for name in names:
        # Remove non-alphabetic characters
        clean_name = ''.join(filter(str.isalpha, name))
        # Correct common OCR errors
        clean_name = clean_name.replace('0', 'O').replace('1', 'I')
        # Validate against known names if provided
        if known_names and clean_name in known_names:
            cleaned_names.append(clean_name)
        elif not known_names:
            cleaned_names.append(clean_name)
    return cleaned_names

def update_working_therapists_list(working_therapists, new_names):
    """
    Update the working therapists list with new names.
    """
    for name in new_names:
        if name not in working_therapists:
            working_therapists.append(name)
    return working_therapists

def main():
    therapists = ["Tracy", "Bakia", "Alex", "Jordan", "Sam", "Chris", "Taylor"]
    known_names = therapists  # Assuming the list of therapists is the list of known names
    image_path = 'path_to_schedule.jpg'
    extracted_text = extract_text_from_image(image_path)
    parsed_names = parse_extracted_text(extracted_text)
    cleaned_names = clean_names(parsed_names, known_names)
    working_therapists = update_working_therapists_list([], cleaned_names)

    special_case = input("Is there a therapist that has called out or requests the cancellation for tonight? (Enter name or 'no'): ")
    if special_case.lower() != 'no':
        cancelled_info, updated_list = update_cancellation_list(working_therapists, therapists, sick_therapist=special_case)
    else:
        cancelled_info, updated_list = update_cancellation_list(working_therapists, therapists)
    print(cancelled_info)
    print("Updated therapists list:", updated_list)

if __name__ == "__main__":
    main()